# Import

In [ ]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import os
import math
from pysentimiento import create_analyzer
from transformers import pipeline


# Read Data

In [ ]:
import pandas as pd

# Read the main general tweets file (TSV format)
df = pd.read_csv('general_tweets.tsv', sep='\t')

# # Define the folder path and list of CSV file names
# output_folder = '/home/noaai/HCP_State_of_Mind_covid_19/new_code_for_artical/general_new/'
# file_names = [
#     'job1_tweets_emotion_pysentimento.csv',
#     'job2_tweets_emotion_pysentimento.csv',
#     'job3_tweets_emotion_pysentimento.csv',
#     'job4_tweets_emotion_pysentimento.csv',
#     'job5_tweets_emotion_pysentimento.csv',
#     'job6_tweets_emotion_pysentimento.csv',
#     'job7_tweets_emotion_pysentimento.csv',
#     'job8_tweets_emotion_pysentimento.csv',
#     'job9_tweets_emotion_pysentimento.csv',
#     'job10_tweets_emotion_pysentimento.csv',
#     'job11_tweets_emotion_pysentimento.csv',
#     'job12_tweets_emotion_pysentimento.csv',
#     'job13_tweets_emotion_pysentimento.csv',
# ]
#
# # Read and concatenate all emotion CSVs into a single DataFrame
# emotion_dfs = [pd.read_csv(f"{output_folder}{file}") for file in file_names]
# general = pd.concat(emotion_dfs, ignore_index=True)


In [5]:
# Remove rows where 'full_text' is NaN
df = df[df['full_text'].notna()]

# Remove rows where 'full_text' is an empty string or contains only whitespace
df = df[df['full_text'].str.strip() != '']


(545276, 8)

In [9]:
# Reset index to ensure clean row access
df = df.reset_index(drop=True)

# Define constants
CHUNK_SIZE = 10_000  # Number of rows per chunk
OUTPUT_DIR = '/home/noaai/HCP_State_of_Mind_covid_19/new_code_for_artical/general_new'

# Initialize the emotion analyzer
emotion_analyzer = create_analyzer(task="emotion", lang="en")

# Loop over the DataFrame in chunks
for chunk_start in range(0, len(df), CHUNK_SIZE):
    chunk_end = chunk_start + CHUNK_SIZE
    chunk_df = df.iloc[chunk_start:chunk_end]

    processed_data = []

    for i, row in chunk_df.iterrows():
        if i % 1000 == 0:
            print(f'Processing row {i}')

        text = row['full_text']

        try:
            result = emotion_analyzer.predict(text)
            processed_data.append([
                row['screen_name'],
                row['created_at'],
                row['name'],
                row['uid'],
                str(row['id_str']),
                result.probas['joy'],
                result.probas['sadness'],
                result.probas['anger'],
                result.probas['surprise'],
                result.probas['disgust'],
                result.probas['fear'],
                result.probas['others'],
                text
            ])
        except Exception as e:
            # Optionally log the error
            continue

    # Create a DataFrame from processed rows
    result_df = pd.DataFrame(processed_data, columns=[
        'screen_name', 'created_at', 'name', 'uid', 'id_str',
        'joy_pys', 'sadness_pys', 'anger_pys', 'surprise_pys', 'disgust_pys', 'fear_pys',
        'others_pys', 'full_text'
    ])

    # Build output file path
    file_number = chunk_start // CHUNK_SIZE + 1
    output_path = os.path.join(OUTPUT_DIR, f'job{file_number}_tweets_emotion_pysentimento.csv')

    # Save to CSV (with header if first write)
    if os.path.exists(output_path):
        result_df.to_csv(output_path, mode='a', header=False, index=False)
    else:
        result_df.to_csv(output_path, index=False)

    print(f'Saved processed chunk to: {output_path}')


Processing row 0
Processing row 1000
Processing row 2000
Processing row 3000
Processing row 4000
Processing row 5000
Processing row 6000
Processing row 7000
Processing row 8000
Processing row 9000
Finished analysis and saved to /home/noaai/HCP_State_of_Mind_covid_19/new_code_for_artical/general_new/job13_tweets_emotion_pysentimento.csv
Processing row 10000
Processing row 11000
Processing row 12000
Processing row 13000
Processing row 14000
Processing row 15000
Processing row 16000
Processing row 17000
Processing row 18000
Processing row 19000
Finished analysis and saved to /home/noaai/HCP_State_of_Mind_covid_19/new_code_for_artical/general_new/job13_tweets_emotion_pysentimento.csv
Processing row 20000
Processing row 21000
Processing row 22000
Processing row 23000
Processing row 24000
Processing row 25000
Processing row 26000
Processing row 27000
Processing row 28000
Processing row 29000
Finished analysis and saved to /home/noaai/HCP_State_of_Mind_covid_19/new_code_for_artical/general_ne

In [9]:
import pandas as pd
import math
import os
df = filtered_df
df = df.dropna(subset=['full_text'])
df = df.reset_index()
# df = df.iloc[400000:]
# df = df.reset_index()
# Assuming df is your large DataFrame
chunk_size = 10000  # Number of rows per chunk
file_num = 18  # File counter

# Assuming create_analyzer is defined and initializes the emotion analyzer
emotion_analyzer = create_analyzer(task="emotion", lang="en")

OUTPUT_FILE_PATH = '/home/noaai/HCP_State_of_Mind_covid_19/new_code_for_artical/general_new'

# Process and save each chunk
for start_row in range(0, len(df), chunk_size):
    end_row = start_row + chunk_size
    chunk_df = df.iloc[start_row:end_row]
    
    data = []
    for index, row in chunk_df.iterrows():
        if index % 1000 == 0:
            print(f'Processing row {index}')
        
        text = row['full_text']
        try:
            analysis_result = emotion_analyzer.predict(text)
            data.append([
                row['screen_name'], row['created_at'], row['name'], row['uid'], 
                row['id_str'], analysis_result.probas['joy'], 
                analysis_result.probas['sadness'], analysis_result.probas['anger'], 
                analysis_result.probas['surprise'], analysis_result.probas['disgust'], 
                analysis_result.probas['fear'], analysis_result.probas['others'], row['full_text']
            ])
        except:
            continue

    result_df = pd.DataFrame(data, columns=[
        'screen_name', 'created_at', 'name', 'uid', 'id_str', 
        'joy_pys', 'sadness_pys', 'anger_pys', 'surprise_pys', 'disgust_pys', 'fear_pys', 
        'others_pys', 'full_text'
    ])
    
    # Convert 'id_str' to string if necessary (ensure correct column name)
    result_df['id_str'] = result_df['id_str'].apply(str)
    
    # Construct file path
    file_path = os.path.join(OUTPUT_FILE_PATH, f'job{file_num}_tweets_emotion_pysentimento.csv')
    
    # Save to CSV, append if file exists, else write with header
    if os.path.isfile(file_path):
        result_df.to_csv(file_path, mode='a', header=False, index=False)
    else:
        result_df.to_csv(file_path, index=False)
    
    print(f'Finished analysis and saved to {file_path}')
    # file_num += 1


Processing row 0
Processing row 1000
Processing row 2000
Processing row 3000
Processing row 4000
Processing row 5000
Processing row 6000
Processing row 7000
Processing row 8000
Processing row 9000
Finished analysis and saved to /home/noaai/HCP_State_of_Mind_covid_19/new_code_for_artical/general_new/job18_tweets_emotion_pysentimento.csv
Processing row 10000
Processing row 11000
Processing row 12000
Processing row 13000
Processing row 14000
Processing row 15000
Processing row 16000
Processing row 17000
Processing row 18000
Processing row 19000
Finished analysis and saved to /home/noaai/HCP_State_of_Mind_covid_19/new_code_for_artical/general_new/job18_tweets_emotion_pysentimento.csv
Processing row 20000
Processing row 21000
Processing row 22000
Processing row 23000
Processing row 24000
Processing row 25000
Processing row 26000
Processing row 27000
Processing row 28000
Processing row 29000
Finished analysis and saved to /home/noaai/HCP_State_of_Mind_covid_19/new_code_for_artical/general_ne

In [13]:
row['full_text']

nan